link for the material: https://nlp.stanford.edu/IR-book/html/htmledition/evaluation-of-ranked-retrieval-results-1.html

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
def mask_relevance(search_result, relevance):
    N = len(search_result)
    relevance_mask = np.zeros(N)
    relevance_mask[(np.array(relevance) - 1)[:, 0]] = 1
    relevance_mask = relevance_mask[np.array(search_result) - 1]
    return relevance_mask

def precision(search_result, relevance):
    N = len(search_result)
    relevance_mask = mask_relevance(search_result, relevance)
    precision = np.zeros(N)
    for p in range(N):
        precision[p] = relevance_mask[:(1 + p)].sum() / (1 + p)
    return precision

def recall(search_result, relevance):
    N = len(search_result)
    relevance_mask = mask_relevance(search_result, relevance)
    M = relevance_mask.sum()
    r = np.zeros(N)
    for p in range(N):
        r[p] = relevance_mask[:(1 + p)].sum()
    return np.divide(r, M).astype(np.float64)

def interpolate11(precision: np.ndarray, recall: np.ndarray):
    p_inter = np.zeros(11)
    for i, r in enumerate(np.linspace(0, 1, num=11)):
        p_inter[i] = precision[recall >= r].max()
    return p_inter

def eleven_points_interpolated_avg(search_results, relevance, plot=True):
    zipped = [(search_results[i], relevance[i + 1]) for i in range(len(search_results))]
    p_inter = []

    for result, relev in zipped:
        p = precision(result, relev)
        r = recall(result, relev)
        p_inter.append(interpolate11(p, r))

    result = np.mean(np.array(p_inter), axis=0)

    if plot:
        sns.lineplot(x=np.linspace(0, 1, 11), y=result)
        plt.show()

    return result